<a href="https://colab.research.google.com/github/quangnguyen-james/Datamining-Technical/blob/main/online_retail_fpgrowth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import fpgrowth, association_rules
from mlxtend.preprocessing import TransactionEncoder
import random

In [5]:
process_data = pd.read_excel('/content/Online Retail.xlsx')
process_data.info()
#bread_data.head(50)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [6]:
#Kiểm tra duplicate
print(process_data.duplicated().sum())
# Lấy ra 10 bản ghi bị duplicate
duplicates = process_data[process_data.duplicated()].head(10)
print(duplicates)

5268
    InvoiceNo StockCode                        Description  Quantity  \
517    536409     21866        UNION JACK FLAG LUGGAGE TAG         1   
527    536409     22866      HAND WARMER SCOTTY DOG DESIGN         1   
537    536409     22900    SET 2 TEA TOWELS I LOVE LONDON          1   
539    536409     22111       SCOTTIE DOG HOT WATER BOTTLE         1   
555    536412     22327  ROUND SNACK BOXES SET OF 4 SKULLS         1   
587    536412     22273               FELTCRAFT DOLL MOLLY         1   
589    536412     22749  FELTCRAFT PRINCESS CHARLOTTE DOLL         1   
594    536412     22141     CHRISTMAS CRAFT TREE TOP ANGEL         1   
598    536412     21448          12 DAISY PEGS IN WOOD BOX         1   
600    536412     22569        FELTCRAFT CUSHION BUTTERFLY         2   

            InvoiceDate  UnitPrice  CustomerID         Country  
517 2010-12-01 11:45:00       1.25     17908.0  United Kingdom  
527 2010-12-01 11:45:00       2.10     17908.0  United Kingdom  
537 201

In [7]:
#Xóa dữ liệu có InvoiceNo là NaN/ NUll
process_data = process_data.dropna(subset=['InvoiceNo'])
process_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [8]:
# Xóa dữ liệu trùng
process_data = process_data.drop_duplicates()
#Kiểm tra trùng
print(process_data.duplicated().sum())


0


In [9]:
# Số sample còn lại
print(process_data.shape)
process_data.head(10)

(536641, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom


In [10]:
print((process_data['Quantity'] <= 0).sum())

10587


In [11]:
#Chỉ lấy dữ liệu có quantity > 0
process_data = process_data[process_data['Quantity'] > 0]
print(process_data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 526054 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    526054 non-null  object        
 1   StockCode    526054 non-null  object        
 2   Description  525462 non-null  object        
 3   Quantity     526054 non-null  int64         
 4   InvoiceDate  526054 non-null  datetime64[ns]
 5   UnitPrice    526054 non-null  float64       
 6   CustomerID   392732 non-null  float64       
 7   Country      526054 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 36.1+ MB
None


In [ ]:
# Loại bỏ các mẫu dữ liệu bắt đầu là ký tự
# Giữ lại các dòng mà InvoiceNo bắt đầu bằng số (0-9)
# Lọc những dòng có InvoiceNo không bị NaN
invoice_notna = process_data[process_data['InvoiceNo'].notna()]

# Giữ lại các dòng bắt đầu bằng số
process_data = invoice_notna[invoice_notna['InvoiceNo'].astype(str).str.match(r'^\d')]
process_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 531282 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    531282 non-null  object        
 1   StockCode    531282 non-null  object        
 2   Description  530690 non-null  object        
 3   Quantity     531282 non-null  int64         
 4   InvoiceDate  531282 non-null  datetime64[ns]
 5   UnitPrice    531282 non-null  float64       
 6   CustomerID   397924 non-null  float64       
 7   Country      531282 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 36.5+ MB


In [12]:
#Đếm số loại sản phẩm (Item)
print(process_data['StockCode'].value_counts())
#Kiểm tra có bao nhiêu sản phẩm (Item)
print(process_data['StockCode'].nunique())
print(process_data['StockCode'].unique())

StockCode
85123A    2258
85099B    2112
22423     2009
47566     1700
20725     1582
          ... 
84550        1
23602        1
23664        1
47591b       1
23843        1
Name: count, Length: 3941, dtype: int64
3941
['85123A' 71053 '84406B' ... '90214U' '47591b' 23843]


In [13]:
#Gộp Items trong cùng 1 giao dịch
transactions = process_data.groupby('InvoiceNo')['Description'].apply(list)

# Chuyển tất cả phần tử trong mỗi transaction thành chuỗi
transactions_str = [[str(item) for item in transaction] for transaction in transactions]
print(transactions)


InvoiceNo
536365     [WHITE HANGING HEART T-LIGHT HOLDER, WHITE MET...
536366     [HAND WARMER UNION JACK, HAND WARMER RED POLKA...
536367     [ASSORTED COLOUR BIRD ORNAMENT, POPPY'S PLAYHO...
536368     [JAM MAKING SET WITH JARS, RED COAT RACK PARIS...
536369                            [BATH BUILDING BLOCK WORD]
                                 ...                        
581586     [LARGE CAKE STAND  HANGING STRAWBERY, SET OF 3...
581587     [CIRCUS PARADE LUNCH BOX , PLASTERS IN TIN CIR...
A563185                                    [Adjust bad debt]
A563186                                    [Adjust bad debt]
A563187                                    [Adjust bad debt]
Name: Description, Length: 20728, dtype: object


In [14]:
# 2. Chuyển dữ liệu sang định dạng one-hot encoding
# One-hot encoding
te = TransactionEncoder()
te_ary = te.fit(transactions_str).transform(transactions_str)

# Tạo DataFrame
df = pd.DataFrame(te_ary, columns=te.columns_)

# In kết quả
df.info()
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20728 entries, 0 to 20727
Columns: 4078 entries,  4 PURPLE FLOCK DINNER CANDLES to wrongly sold (22719) barcode
dtypes: bool(4078)
memory usage: 80.6 MB
    4 PURPLE FLOCK DINNER CANDLES   50'S CHRISTMAS GIFT BAG LARGE  \
0                           False                           False   
1                           False                           False   
2                           False                           False   
3                           False                           False   
4                           False                           False   

    DOLLY GIRL BEAKER   I LOVE LONDON MINI BACKPACK  \
0               False                         False   
1               False                         False   
2               False                         False   
3               False                         False   
4               False                         False   

    I LOVE LONDON MINI RUCKSACK   NINE DRAWER OFFIC

In [19]:
# 3. Áp dụng thuật toán FP-Growth
min_support = 0.050  # Ngưỡng support tối thiểu: 5%
frequent_itemsets = fpgrowth(df, min_support=min_support, use_colnames=True)

In [24]:
print(frequent_itemsets.shape)
print(frequent_itemsets.head())

(15195, 2)
    support                               itemsets
0  0.109031   (WHITE HANGING HEART T-LIGHT HOLDER)
1  0.022144  (KNITTED UNION FLAG HOT WATER BOTTLE)
2  0.020697       (RED WOOLLY HOTTIE WHITE HEART.)
3  0.018333         (SET 7 BABUSHKA NESTING BOXES)
4  0.014811                  (WHITE METAL LANTERN)


In [25]:
# 4. Tạo luật kết hợp
min_confidence = 0.7  # Ngưỡng confidence tối thiểu:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)

In [26]:
# 5. In kết quả
print("Frequent Itemsets:")
print(frequent_itemsets.sort_values(by='support', ascending=False).head(10))
print("\nAssociation Rules:")
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10))

Frequent Itemsets:
       support                              itemsets
0     0.109031  (WHITE HANGING HEART T-LIGHT HOLDER)
122   0.100926             (JUMBO BAG RED RETROSPOT)
421   0.095957            (REGENCY CAKESTAND 3 TIER)
983   0.081339                       (PARTY BUNTING)
53    0.075453             (LUNCH BAG RED RETROSPOT)
8     0.070195       (ASSORTED COLOUR BIRD ORNAMENT)
1148  0.066818   (SET OF 3 CAKE TINS PANTRY DESIGN )
54    0.063682     (PACK OF 72 RETROSPOT CAKE CASES)
229   0.061415             (LUNCH BAG  BLACK SKULL.)
107   0.060257     (NATURAL SLATE HEART CHALKBOARD )

Association Rules:
                                         antecedents  \
0  (HOT WATER BOTTLE I AM SO POORLY, RED WOOLLY H...   
1                       (POPPY'S PLAYHOUSE BEDROOM )   
2                        (POPPY'S PLAYHOUSE KITCHEN)   
3  (JAM MAKING SET WITH JARS, JUMBO BAG PINK POLK...   
4  (PAPER CHAIN KIT 50'S CHRISTMAS , ALARM CLOCK ...   
5  (ALARM CLOCK BAKELIKE GREEN, REGENCY CA

In [ ]:
# 6. Lưu dữ liệu giả lập và kết quả (tùy chọn)
df_transactions = pd.DataFrame(transactions)
df_transactions.to_csv('simulated_transactions.csv', index=False)
frequent_itemsets.to_csv('frequent_itemsets.csv', index=False)
rules.to_csv('association_rules.csv', index=False)